In [2]:
from datasets import load_dataset

In [3]:
ds = load_dataset("squad")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
ds["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [18]:
inputs = tokenizer(
    ds["train"][2:6]["question"],
    ds["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)


In [85]:
test_data = ds["train"][2:6]
answers = test_data["answers"]
start_positions = []
end_positions = []

In [100]:
answers

[{'text': ['the Main Building'], 'answer_start': [279]},
 {'text': ['a Marian place of prayer and reflection'], 'answer_start': [381]},
 {'text': ['a golden statue of the Virgin Mary'], 'answer_start': [92]},
 {'text': ['September 1876'], 'answer_start': [248]}]

In [101]:
len(inputs["offset_mapping"]), len(inputs['input_ids'])

(19, 19)

In [59]:
ds["train"][2]

{'id': '5733be284776f41900661180',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'answers': {'text': ['the Main Building'], 'answer_start': [279]}}

In [54]:
len(tokenizer.tokenize(ds["train"][2]['context'])),tokenizer.tokenize(ds["train"][2]['context'])[:2]

(163, ['Architectural', '##ly'])

In [66]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

Input:

The tokenizer splits each question, context into max length and overlfows if more than max length. Here the same context can be split into multiple records

Start index of the character where answer begins

Goal: 

From the context find start index of the token corresponding to the answer in the context and the end index of the token corresponding to where the answer ends


Solution:

2. Take the length of the answer and find the last character index from start character index given for the answer
3. Offset mapping has start character and end characters for each token generated from input ids

first - 250, last - 270

(1,12),(13,14),..(247,249),(249,260),(261,275)

5. For each offset mapping token, check if the first character index of the token  == start character string then choose the index for that token within the input id
6. Similarly check if the last character index of the token >= last character index then return the index of the token within the input ID

Input ID is the combination of both question + context. Find the context before doing steps 3,4,5






In [5]:
def getstartandendtokens(inputs, answers):
        
    start_ans_index = []
    end_ans_index = []
    for index, offset in enumerate(inputs["offset_mapping"]):
        split = inputs["overflow_to_sample_mapping"][index]
        answer = answers[split]
        start_char_ans = answer['answer_start'][0]
        end_char_ans = start_char_ans+len(answer['text'][0])
       
        sequence_ids = inputs.sequence_ids(index)
    
        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        
        if offset[context_start][0] > start_char_ans or offset[context_end][1] < end_char_ans:
            start_ans_index.append(0)
            end_ans_index.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char_ans:
                idx += 1
            start_ans_index.append(idx - 1)
    
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char_ans:
                idx -= 1
            end_ans_index.append(idx + 1)
    return start_ans_index,end_ans_index
    
            
    
    
    

In [126]:
print(getstartandendtokens(inputs))

19
([83, 51, 19, 0, 0, 64, 27, 0, 34, 0, 0, 0, 67, 34, 0, 0, 0, 0, 0], [85, 53, 21, 0, 0, 70, 33, 0, 40, 0, 0, 0, 68, 35, 0, 0, 0, 0, 0])


In [6]:
def preprocesstrainingdata(example):
    inputs = tokenizer(example["question"],
                      example["context"],
                      truncation = "only_second",
                      max_length = 384,
                      stride=128,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
                       padding="max_length",)
    answers = example["answers"]
    inputs["start_index"], inputs["end_index"] = getstartandendtokens(inputs,answers)
    return inputs
    


In [123]:
ds["train"].column_names

['id', 'title', 'context', 'question', 'answers']

In [7]:
tokenized_train_ds = ds["train"].map(preprocesstrainingdata, batched=True, remove_columns = ds["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [147]:
len(tokenized_train_ds) - len(ds["train"])

1130

<b>Validation Set</b>

In [151]:
ds["validation"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

In [21]:
def preprocessvalidationset(example):
    
    inputs = tokenizer(example["question"],
                            example["context"],truncation ="only_second",
                  max_length = 384,
                      stride=128,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
                       padding="max_length",)
    validation_sample = example["id"]
    validation_id = []
    context_id = []
    for item in range(len(inputs['input_ids'])):
        offset_id = inputs['overflow_to_sample_mapping'][item]
        validation_id.append(validation_sample[offset_id])
        offset = inputs['offset_mapping'][item]
        sequence = inputs.sequence_ids(item)
        inputs['offset_mapping'][item] = [value if sequence[i] == 1 else None for  i, value in enumerate(offset)]
    inputs["validation_id"] = validation_id
    return inputs


In [22]:
tokenized_ds_validation = ds["validation"].map(preprocessvalidationset, batched = True, remove_columns = ds["validation"].column_names)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [23]:
tokenized_ds_validation

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping', 'validation_id'],
    num_rows: 10822
})

In [24]:
small_eval_set = ds["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocessvalidationset,
    batched=True,
    remove_columns=ds["validation"].column_names,
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [25]:
eval_set

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping', 'validation_id'],
    num_rows: 100
})

In [29]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["validation_id", "offset_mapping","overflow_to_sample_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}


In [30]:
batch

{'input_ids': tensor([[  101,  5979,  4279,  ...,     0,     0,     0],
         [  101,  5979,  4279,  ...,     0,     0,     0],
         [  101,  2777,  1225,  ...,     0,     0,     0],
         ...,
         [  101,  1327,  1160,  ...,     0,     0,     0],
         [  101,  2627,  3012,  ...,     0,     0,     0],
         [  101,  2627, 21188,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [31]:
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [33]:
outputs.start_logits.shape

torch.Size([100, 384])

In [51]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [36]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["validation_id"]].append(idx)

In [40]:
example_to_features['56be4db0acb8001400a502ec']

[0]

In [41]:
len(small_eval_set), len(eval_set)

(100, 100)

In [42]:
small_eval_set

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 100
})

In [47]:
small_eval_set, eval_set

(Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 100
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping', 'validation_id'],
     num_rows: 100
 }))

In [73]:
import numpy as np
n_best = 20
max_answer_length =30
predicted_answers = []
for item in small_eval_set:
    id, context, answers = item['id'], item['context'], item['answers']
    answers = []
    for tok_index in example_to_features[id]:
        offset = eval_set["offset_mapping"][tok_index]
        start_logit = start_logits[tok_index]
        end_logit= end_logits[tok_index]
        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        #print(start_indexes, end_indexes)
        for start in start_indexes:
            for end in end_indexes:
                if offset[start] == None or offset[end]  == None:
                    continue
                if end < start or end - start + 1 > max_answer_length:
                    continue
                answers.append({
                    "text":context[offset[start][0]:offset[end][1]],
                "score":start_logit[start]+end_logit[end]}
                
                )
    best_answer = max(answers, key = lambda x: x["score"])
    predicted_answers.append({"id": id, "prediction_text": best_answer["text"]})

In [74]:
predicted_answers

[{'id': '56be4db0acb8001400a502ec', 'prediction_text': 'Denver Broncos'},
 {'id': '56be4db0acb8001400a502ed', 'prediction_text': 'Carolina Panthers'},
 {'id': '56be4db0acb8001400a502ee',
  'prediction_text': "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California"},
 {'id': '56be4db0acb8001400a502ef', 'prediction_text': 'Carolina Panthers'},
 {'id': '56be4db0acb8001400a502f0', 'prediction_text': 'gold'},
 {'id': '56be8e613aeaaa14008c90d1', 'prediction_text': 'golden anniversary'},
 {'id': '56be8e613aeaaa14008c90d2', 'prediction_text': 'February 7, 2016'},
 {'id': '56be8e613aeaaa14008c90d3',
  'prediction_text': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference'},
 {'id': '56bea9923aeaaa14008c91b9', 'prediction_text': 'golden anniversary'},
 {'id': '56bea9923aeaaa14008c91ba',
  'prediction_text': 'American Football Conference'},
 {'id': '56bea9923aeaaa14008c9